**Template 1: FRAUD_CLASSIFIER**
Purpose: Binary classification with confidence

**What you need to figure out:**
- What role description sets fraud detection context?
- What constraints prevent hallucination?
- What output format ensures parseability?
- How to handle "insufficient data" cases?

In [114]:
import ollama

FRAUD_CLASSIFIER_SYSTEM_TEMPLATE ="""
--- SYSTEM INSTRUCTIONS ---
Role: You are a Financial Fraud Investigator with American Express, specialized in identifying Fraud Transactions.
Task: Analyse transaction data for Fraud based on provided facts only.

--- RULES ---
1. Flag as "Insufficient Data" if Transaction Id, Card Number, Time of transaction, Amount or Location are missing in input.
2. If amount is less than 0, flag as "Suspicious".
3. If amount is greater than 1000, flag as "Suspicious"

---OUTPUT FORMAT---
Respond only in JSON format with fields:
Transaction Id : txn_id
Transaction Fraud Category : txn_fraud_category
Reasoning : reason
"""

FRAUD_CLASSIFIER_USER_TEMPLATE ="""
--- CONTEXT / DATA ---
Required details:
    Transaction Id: {txn_id}
    Card Number: {card_number}
    Time of transaction: {timestamp}
    Amount: {amount}
    Merchant: {merchant}
    Category:{category}
    Location: {location}
"""

In [115]:
# import ollama
#
# FRAUD_CLASSIFIER_SYSTEM_TEMPLATE = """
# You are a Financial Fraud Investigator. Analyze transactions for fraud.
#
# RULES:
# 1. If any of these fields are missing or empty, respond with "Insufficient Data": txn_id, card_number, timestamp, amount, location
# 2. If amount < 0, flag as "Suspicious"
# 3. If amount > 1000, flag as "Suspicious"
# 4. Otherwise, flag as "Legitimate"
#
# OUTPUT FORMAT (JSON only):
# {
#   "Transaction Id": "string",
#   "Transaction Fraud Category": "Legitimate | Suspicious | Insufficient Data",
#   "Reasoning": "string"
# }
# """
#
# FRAUD_CLASSIFIER_USER_TEMPLATE = """
# Analyze this transaction:
#
# Transaction Id: {txn_id}
# Card Number: {card_number}
# Timestamp: {timestamp}
# Amount: {amount}
# Merchant: {merchant}
# Category: {category}
# Location: {location}
# """

In [116]:
user_message = FRAUD_CLASSIFIER_USER_TEMPLATE.format(
    txn_id="TXN_104",
    card_number="1234-5678-9012-3456",
    timestamp="2023-10-23 8:15:00",
    amount=2.50,
    merchant="TFL",
    category="Transport",
    location="London, UK"
)

# Debug: Print the formatted message to see what the model receives
print("DEBUG: Formatted user message:")
print(user_message)
print("\n" + "="*60 + "\n")

payload = [
    {"role": "system", "content": FRAUD_CLASSIFIER_SYSTEM_TEMPLATE},
    {"role": "user", "content": user_message}
]

# Add options for deterministic/consistent responses
options = {
    "temperature": 0.0,  # Set to 0 for maximum consistency
    "top_p": 1.0,
    "seed": 42  # Fixed seed for reproducibility
}

response = ollama.chat(
    model="llama3.2:latest", 
    stream=False, 
    # format='json',
    messages=payload,
    # options=options  # Add this!
)
print("Model response:")
print(response.message.content)

DEBUG: Formatted user message:

--- CONTEXT / DATA ---
Required details:
    Transaction Id: TXN_104
    Card Number: 1234-5678-9012-3456
    Time of transaction: 2023-10-23 8:15:00
    Amount: 2.5
    Merchant: TFL
    Category:Transport
    Location: London, UK



Model response:
{
    "txn_id": "TXN_104",
    "txn_fraud_category": "",
    "reason": ""
}


**Improved Template**

In [117]:
FRAUD_CLASSIFIER_SYSTEM_TEMPLATE_NEW = """
--- SYSTEM INSTRUCTIONS ---
Role: You are a Senior Risk Analyst for a financial institution.
Task: Evaluate the provided transaction metadata for potential fraud indicators.
Constraints:
1. PII SAFETY: Do NOT ask for or output real names or full card numbers. Use IDs only.
2. NO HALLUCINATION: If critical data (Amount, Merchant, Time) is missing, classify as "INSUFFICIENT_DATA".
3. REASONING: You must think step-by-step before assigning a label.

--- RULES ---
1. THRESHOLDS:
   - Amount > 5000 USD is "HIGH_VALUE".
   - Amount < 0 is "INVALID".
2. LOGIC:
   - If Location matches "known_fraud_hotspots" (e.g., specific high-risk regions defined in policy), increase suspicion.
   - If Amount is "HIGH_VALUE" AND Time is "Late Night" (00:00-04:00), flag as SUSPICIOUS.

--- OUTPUT FORMAT ---
You must respond with a valid JSON object ONLY. No markdown, no conversational text.
Allowed Categories: ["LEGITIMATE", "SUSPICIOUS", "INSUFFICIENT_DATA"]

{
    "transaction_id": "str",
    "risk_assessment": {
        "classification": "ENUM",
        "confidence_score": "float (0.0 to 1.0)",
        "primary_risk_factor": "str or null"
    },
    "explanation": "concise reasoning string"
}
"""
FRAUD_CLASSIFIER_USER_TEMPLATE_NEW ="""
--- CONTEXT / DATA ---
Transaction ID: {txn_id}
Hashed Card ID: {hashed_card_id}
Timestamp: {timestamp}
Amount: {amount} {currency}
Merchant: {merchant}
Merchant Category: {merchant_category}
Location: {location_city}, {location_country}
"""

In [118]:
user_message = FRAUD_CLASSIFIER_USER_TEMPLATE_NEW.format(
    txn_id="TXN_104",
    hashed_card_id="1234-5678-9012-3456",
    timestamp="2023-10-23T08:15:00Z",
    amount=2.50,
    merchant="TFL",
    merchant_category="Transport",
    location_city="London",
    location_country="UK",
    currency="USD"
)

# Debug: Print the formatted message to see what the model receives
print("DEBUG: Formatted user message:")
print(user_message)
print("\n" + "="*60 + "\n")

payload = [
    {"role": "system", "content": FRAUD_CLASSIFIER_SYSTEM_TEMPLATE_NEW},
    {"role": "user", "content": user_message}
]

response = ollama.chat(model="llama3.2:latest", stream=False, format='json', messages=payload)
print("Model response:")
print(response.message.content)

DEBUG: Formatted user message:

--- CONTEXT / DATA ---
Transaction ID: TXN_104
Hashed Card ID: 1234-5678-9012-3456
Timestamp: 2023-10-23T08:15:00Z
Amount: 2.5 USD
Merchant: TFL
Merchant Category: Transport
Location: London, UK



Model response:
{
    "transaction_id": "TXN_104",
    "risk_assessment": {
        "classification": "LEGITIMATE",
        "confidence_score": 0.99,
        "primary_risk_factor": null
    },
    "explanation": "The transaction is legitimate as it involves a small amount (< 5000 USD), takes place during normal hours (08:15, not late night), and is from a known merchant (TFL) in a standard location (London, UK)"
}


In [120]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",  # Add /v1 for OpenAI compatibility
    api_key="ollama"
)

response = client.chat.completions.create(
    model="llama3.2:latest", 
    stream=False, 
    messages=payload
)
print(response.choices[0].message.content)

{
    "transaction_id": "TXN_104",
    "risk_assessment": {
        "classification": "LEGITIMATE",
        "confidence_score": 0.95,
        "primary_risk_factor": null
    },
    "explanation": "Location is in a known legitimate area (London, UK), and the merchant category is transport, making this transaction normal."
